In [ ]:
import ROOT
from pyroot_cms_scripts import CMS_style
import os

In [ ]:
configs = [
    "wv_BDTG6",
    "wjj_BDTG6",
]
bdt_files = {
    c: ROOT.TFile.Open(f"{c}/tmva_output.root") for c in configs
}

plots_dir = "tmva_plots"

In [ ]:
bdt_outputs = {
    c: f.Get(f"{c}/Method_BDT/BDT") for c, f in bdt_files.items()
}

In [ ]:
ROOT.gROOT.ForceStyle()
ROOT.gStyle.SetOptStat(0)
ROOT.gStyle.SetPalette(ROOT.kRainBow)

In [ ]:
hists = {}
roc = {}
for c, f in bdt_outputs.items():
    hists[f"{c}_mva_s"] = f.Get("MVA_BDT_S")
    hists[f"{c}_mva_b"] = f.Get("MVA_BDT_B")
    hists[f"{c}_mva_s_train"] = f.Get("MVA_BDT_Train_S")
    hists[f"{c}_mva_b_train"] = f.Get("MVA_BDT_Train_B")

    roc[f"{c}_effBvsS"] = f.Get("MVA_BDT_effBvsS")
    roc[f"{c}_effBvsS_train"] = f.Get("MVA_BDT_trainingEffBvsS")

In [ ]:
draw_mva = """
CMS_style.cd()
canvas = ROOT.TCanvas("", "", 600, 600)
legend = ROOT.TLegend(0.6, 0.8, 0.93, 0.92, f"{c}")
legend.SetNColumns(2)
legend.SetTextFont(42)
legend.SetBorderSize(1)

hs = hists[f"{c}_mva_s"]
hst = hists[f"{c}_mva_s_train"]
ks_signal = hs.KolmogorovTest(hst, "X")
print(ks_signal)

hb = hists[f"{c}_mva_b"]
hbt = hists[f"{c}_mva_b_train"]
ks_bkg = hb.KolmogorovTest(hbt, "X")
print(ks_bkg)

hs.SetLineColor(ROOT.kBlue)
hs.SetFillColorAlpha(ROOT.kBlue, 0.5)
hs.SetFillStyle(1001)
hst.SetMarkerColor(ROOT.kBlue)
hst.SetMarkerStyle(20)

hb.SetLineColor(ROOT.kRed)
hb.SetFillColorAlpha(ROOT.kRed, 0.5)
hb.SetFillStyle(1001)
hbt.SetMarkerColor(ROOT.kRed)
hbt.SetMarkerStyle(20)

legend.AddEntry(hs, "Signal", "lf")
legend.AddEntry(hst, "Signal (Train)", "ep")
#legend.AddEntry("", f"KS-S {ks_signal}", "")

legend.AddEntry(hb, "Bkg", "lf")
legend.AddEntry(hbt, "Bkg (Train)", "ep")
#legend.AddEntry("", f"KS-B {ks_bkg}", "")

frame = max(hs, hst, hb, hbt, key=lambda x: x.GetMaximum()).Clone("frame")
frame.SetMaximum(1.4 * frame.GetMaximum())
frame.SetTitle(f"{c};BDT Score;")
frame.Draw("axis")

hs.Draw("hist same")
hst.Draw("x0 p same")
hb.Draw("hist same")
hbt.Draw("x0 p same")

legend.Draw()
canvas.Draw()
os.makedirs(f"{plots_dir}", exist_ok=True)
canvas.SaveAs(f"{plots_dir}/{c}.pdf")
canvas.SaveAs(f"{plots_dir}/{c}.png")
"""

In [ ]:
c = "wv_BDTG6"
exec(draw_mva)

In [ ]:
c = "wjj_BDTG6"
exec(draw_mva)

In [ ]:
draw_roc = """
CMS_style.cd()
canvas = ROOT.TCanvas()
frame = canvas.DrawFrame(0.0, 0.0, 1.0, 1.0, ";Sig Eff.;Bkg Eff.")
frame.Draw()
legend = ROOT.TLegend(0.18, 0.87, 0.4, 0.92)
legend.SetBorderSize(1)
for c in configs:
    r = roc[f"{c}_effBvsS"]
    r.Draw("same C PLC")
    auc = 1 - (r.Integral()/100)
    legend.AddEntry(r, f"{c} ({auc:.3f})", "l")
legend.Draw()
canvas.SetLogy(0)
canvas.SetGrid()
canvas.Draw()
os.makedirs(f"{plots_dir}", exist_ok=True)
canvas.SaveAs(f"{plots_dir}/roc.pdf")
canvas.SaveAs(f"{plots_dir}/roc.png")
"""
exec(draw_roc)